# Geographical and Temporal Aggregation (Tourism)

> Geographical and Temporal Hierarchical Forecasting on Australian Tourism Data

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

In this notebook we present an example on how to use `HierarchicalForecast` to produce coherent forecasts between geographical levels. We will use the classic Australian Domestic Tourism (`Tourism`) dataset, which contains monthly time series of the number of visitors to each state of Australia.

We will first load the `Tourism` data and produce base forecasts using an `ETS` model from `StatsForecast`, and then reconciliate the forecasts with several reconciliation algorithms from `HierarchicalForecast`. Finally, we show the performance is comparable with the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html) which uses the R package [fable](https://github.com/tidyverts/fable).

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df["Year"] = Y_df["ds"].dt.year
Y_df.head()

,Country,Region,State,Purpose,ds,y,Year
0,Australia,Adelaide,South Australia,Business,1998-01-01,135.077690,1998
1,Australia,Adelaide,South Australia,Business,1998-04-01,109.987316,1998
2,Australia,Adelaide,South Australia,Business,1998-07-01,166.034687,1998
3,Australia,Adelaide,South Australia,Business,1998-10-01,127.160464,1998
4,Australia,Adelaide,South Australia,Business,1999-01-01,137.448533,1999


## 2. Cross-sectional reconciliation

### 2a. Aggregating the dataset according to cross-sectional hierarchy

The dataset can be grouped in the following non-strictly hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
from hierarchicalforecast.utils import aggregate

In [ ]:
Y_df_cs, S_df_cs, tags_cs = aggregate(Y_df, spec)

In [ ]:
Y_df_cs

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511
3,Australia,1998-10-01,20830.129891
4,Australia,1999-01-01,22087.353380
...,...,...,...
33995,Australia/Western Australia/Experience Perth/V...,2016-10-01,439.699451
33996,Australia/Western Australia/Experience Perth/V...,2017-01-01,356.867038
33997,Australia/Western Australia/Experience Perth/V...,2017-04-01,302.296119
33998,Australia/Western Australia/Experience Perth/V...,2017-07-01,373.442070


In [ ]:
S_df_cs.iloc[:5, :5]

,unique_id,Australia/ACT/Canberra/Business,Australia/ACT/Canberra/Holiday,Australia/ACT/Canberra/Other,Australia/ACT/Canberra/Visiting
0,Australia,1.0,1.0,1.0,1.0
1,Australia/ACT,1.0,1.0,1.0,1.0
2,Australia/New South Wales,0.0,0.0,0.0,0.0
3,Australia/Northern Territory,0.0,0.0,0.0,0.0
4,Australia/Queensland,0.0,0.0,0.0,0.0


In [ ]:
tags_cs['Country/Purpose']

array(['Australia/Business', 'Australia/Holiday', 'Australia/Other',
       'Australia/Visiting'], dtype=object)

### 2b. Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_test_df_cs = Y_df_cs.groupby("unique_id", as_index=False).tail(8)
Y_train_df_cs = Y_df_cs.drop(Y_test_df_cs.index)

In [ ]:
Y_train_df_cs.groupby("unique_id").size()

unique_id
Australia                                                72
Australia/ACT                                            72
Australia/ACT/Business                                   72
Australia/ACT/Canberra                                   72
Australia/ACT/Canberra/Business                          72
                                                         ..
Australia/Western Australia/Experience Perth/Other       72
Australia/Western Australia/Experience Perth/Visiting    72
Australia/Western Australia/Holiday                      72
Australia/Western Australia/Other                        72
Australia/Western Australia/Visiting                     72
Length: 425, dtype: int64

### 2c. Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `ETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [ ]:
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], 
                     freq='QS', n_jobs=-1)
Y_hat_df_cs = fcst.forecast(df=Y_train_df_cs, h=8, fitted=True)
Y_fitted_df_cs = fcst.forecast_fitted_values()

### 2d. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_cs = hrec.reconcile(Y_hat_df=Y_hat_df_cs, Y_df=Y_fitted_df_cs, S=S_df_cs, tags=tags_cs)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df_cs.head()

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.911737,25428.089783,25894.399067
1,Australia,2016-04-01,24458.490282,22903.895964,23914.271400,24357.301898
2,Australia,2016-07-01,23974.055984,22412.265739,23428.462394,23865.910647
3,Australia,2016-10-01,24563.454495,23127.349578,24089.845955,24470.782393
4,Australia,2017-01-01,25990.068004,24518.118006,25545.358678,25901.362283


## 3. Temporal reconciliation

### 3a. Aggregating the dataset according to temporal hierarchy

In [ ]:
from hierarchicalforecast.utils import aggregate_temporal

In [ ]:
Y_df_cs["Year"] = Y_df_cs["ds"].dt.year

In [ ]:
spec_temporal = [
    ["Year"],
    ["Year", "ds"],
]

In [ ]:
Y_df_te, S_df_te, tags_te = aggregate_temporal(Y_df_cs, spec_temporal)

In [ ]:
Y_df_te

,unique_id_ds,unique_id,y,ds
0,1998,Australia,84162.347737,1998-01-01
1,1998,Australia/ACT,1852.855000,1998-01-01
2,1998,Australia/ACT/Business,481.394885,1998-01-01
3,1998,Australia/ACT/Canberra,1852.855000,1998-01-01
4,1998,Australia/ACT/Canberra/Business,481.394885,1998-01-01
...,...,...,...,...
42495,2017/2017-10-01,Australia/Western Australia/Experience Perth/O...,87.494916,2017-10-01
42496,2017/2017-10-01,Australia/Western Australia/Experience Perth/V...,455.316702,2017-10-01
42497,2017/2017-10-01,Australia/Western Australia/Holiday,1026.285985,2017-10-01
42498,2017/2017-10-01,Australia/Western Australia/Other,161.087339,2017-10-01


### 2b. Split Train/Test sets

This step is now less obvious, as we have different frequencies as a result of the temporal aggregation. Hence, we generate train- and test sets for every temporal tag and corresponding frequency we've now generated.

It is important that both train- and test sets contain the same timestamps, and that there are no incomplete temporal aggregations [tbc]

In [ ]:
Y_dfs_te = {}
max_date = "2016-12-31"
for tag in tags_te:
    Y_tag = Y_df_te.query("unique_id_ds in @tags_te[@tag]")
    Y_tag_train = Y_tag.query("@pd.to_datetime(ds) <= @max_date")
    Y_tag_test = Y_tag.query("@pd.to_datetime(ds) > @max_date")
    frequency = pd.infer_freq(Y_tag["ds"].unique())
    Y_dfs_te[tag] = {"Y_df": Y_tag_train, "Y_test_df": Y_tag_test, "freq": frequency}

In [ ]:
Y_dfs_te

{'Year': {'Y_df':      unique_id_ds                                          unique_id  \
  0            1998                                          Australia   
  1            1998                                      Australia/ACT   
  2            1998                             Australia/ACT/Business   
  3            1998                             Australia/ACT/Canberra   
  4            1998                    Australia/ACT/Canberra/Business   
  ...           ...                                                ...   
  8070         2016  Australia/Western Australia/Experience Perth/O...   
  8071         2016  Australia/Western Australia/Experience Perth/V...   
  8072         2016                Australia/Western Australia/Holiday   
  8073         2016                  Australia/Western Australia/Other   
  8074         2016               Australia/Western Australia/Visiting   
  
                   y         ds  
  0     84162.347737 1998-01-01  
  1      1852.855000 1998

### 2c. Computing base forecasts

The following cell computes the **base forecasts** for each temporal aggregation in `Y_dfs_te` using the `AutoETS` model. Observe that `Y_hat_df_te` contains the forecasts but they are not coherent.

In [ ]:
Y_hat_dfs_te = []
Y_fitted_dfs_te = []
for tag, Y_tag in Y_dfs_te.items():
    Y_tag_train = Y_tag["Y_df"]
    Y_tag_test = Y_tag["Y_test_df"]
    freq = Y_tag["freq"]
    fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], freq=freq, n_jobs=-1)
    Y_hat_df_te_tag = fcst.forecast(df=Y_tag_train[["ds", "unique_id", "y"]], h=8, fitted=True)
    Y_fitted_df_te_tag = fcst.forecast_fitted_values()
    Y_hat_df_te_tag = Y_hat_df_te_tag.merge(Y_tag_train[["unique_id_ds", "ds", "unique_id"]], on=["ds", "unique_id"], how="left")
    Y_fitted_df_te_tag = Y_fitted_df_te_tag.merge(Y_tag_train[["unique_id_ds", "ds", "unique_id"]], on=["ds", "unique_id"], how="left")
    Y_hat_dfs_te.append(Y_hat_df_te_tag)
    Y_fitted_dfs_te.append(Y_fitted_df_te_tag)
    Y_dfs_te[tag]["Y_hat_df"] = Y_hat_df_te_tag
    Y_dfs_te[tag]["Y_fitted_df"] = Y_fitted_df_te_tag


In [ ]:
Y_hat_df_te_tag

,unique_id,ds,AutoETS,unique_id_ds
0,Australia,2017-01-01,27045.917903,NaN
1,Australia,2017-04-01,25423.019816,NaN
2,Australia,2017-07-01,24938.165386,NaN
3,Australia,2017-10-01,25640.552685,NaN
4,Australia,2018-01-01,27045.917903,NaN
...,...,...,...,...
3395,Australia/Western Australia/Visiting,2017-10-01,757.232213,NaN
3396,Australia/Western Australia/Visiting,2018-01-01,772.284789,NaN
3397,Australia/Western Australia/Visiting,2018-04-01,725.988191,NaN
3398,Australia/Western Australia/Visiting,2018-07-01,691.645476,NaN


In [ ]:
Y_hat_df_te = pd.concat(Y_hat_dfs_te)

In [ ]:
Y_fitted_dfs_te

[                                 unique_id         ds             y  \
 0                                Australia 1998-01-01  84162.347737   
 1                                Australia 1999-01-01  83487.844813   
 2                                Australia 2000-01-01  83951.012082   
 3                                Australia 2001-01-01  84252.302467   
 4                                Australia 2002-01-01  86387.002609   
 ...                                    ...        ...           ...   
 8070  Australia/Western Australia/Visiting 2012-01-01   2018.522124   
 8071  Australia/Western Australia/Visiting 2013-01-01   2074.652920   
 8072  Australia/Western Australia/Visiting 2014-01-01   3011.350654   
 8073  Australia/Western Australia/Visiting 2015-01-01   2989.173687   
 8074  Australia/Western Australia/Visiting 2016-01-01   2976.410922   
 
            AutoETS unique_id_ds  
 0     84053.575611         1998  
 1     84232.129714         1999  
 2     83331.864563         2

In [ ]:
# pd.to_datetime(Y_df_te["ds"], format="mixed")

0       1998-01-01
1       1998-01-01
2       1998-01-01
3       1998-01-01
4       1998-01-01
           ...    
42495   2017-10-01
42496   2017-10-01
42497   2017-10-01
42498   2017-10-01
42499   2017-10-01
Name: ds, Length: 42500, dtype: datetime64[ns]

In [ ]:
Y_df_te

,unique_id_ds,unique_id,y,ds
0,1998,Australia,84162.347737,1998-01-01
1,1998,Australia/ACT,1852.855000,1998-01-01
2,1998,Australia/ACT/Business,481.394885,1998-01-01
3,1998,Australia/ACT/Canberra,1852.855000,1998-01-01
4,1998,Australia/ACT/Canberra/Business,481.394885,1998-01-01
...,...,...,...,...
42495,2017/2017-10-01,Australia/Western Australia/Experience Perth/O...,87.494916,2017-10-01
42496,2017/2017-10-01,Australia/Western Australia/Experience Perth/V...,455.316702,2017-10-01
42497,2017/2017-10-01,Australia/Western Australia/Holiday,1026.285985,2017-10-01
42498,2017/2017-10-01,Australia/Western Australia/Other,161.087339,2017-10-01


In [ ]:
# Y_hat_df_te.merge(Y_df_te[["unique_id_ds", "unique_id", "ds"]], on=["unique_id", "ds"], how="left")

,unique_id,ds,AutoETS,unique_id_ds
0,Australia,2017-01-01,98384.033035,2017
1,Australia,2017-01-01,98384.033035,2017/2017-01-01
2,Australia,2018-01-01,101572.249896,NaN
3,Australia,2019-01-01,101641.968609,NaN
4,Australia,2020-01-01,101486.295068,NaN
...,...,...,...,...
7645,Australia/Western Australia/Visiting,2017-10-01,757.232213,2017/2017-10-01
7646,Australia/Western Australia/Visiting,2018-01-01,772.284789,NaN
7647,Australia/Western Australia/Visiting,2018-04-01,725.988191,NaN
7648,Australia/Western Australia/Visiting,2018-07-01,691.645476,NaN


### 2d. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_te = hrec.reconcile(Y_hat_df=Y_hat_df_te, Y_df=Y_fitted_df_te, S=S_df_te, tags=tags_te)

NameError: name 'Y_fitted_df_te' is not defined

## 4. Evaluation 

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [ ]:
def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

evaluator = HierarchicalEvaluation(evaluators=[rmse, mase])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)

evaluation.columns = ['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

### RMSE

The following table shows the performance measured using RMSE across levels for each reconciliation method.

In [ ]:
evaluation.query('metric == "rmse"')

### MASE


The following table shows the performance measured using MASE across levels for each reconciliation method.

In [ ]:
evaluation.query('metric == "mase"')

### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html). The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![Fable's reconciliation results](./imgs/AustralianDomesticTourism-results-fable.png)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)